In [13]:
import cobra
from cobra.flux_analysis.deletion import single_reaction_deletion
from cobra.flux_analysis.parsimonious import add_pfba

In [66]:
model = cobra.io.read_sbml_model('../genres/97138.36.fbc.sbml')

In [67]:
model

Name,A_97138_36
Memory address,0x07f97b5d95be0
Number of metabolites,1150
Number of reactions,1040
Number of groups,0
Objective expression,1.0*bio1 - 1.0*bio1_reverse_b18f7
Compartments,"Cytosol_0, Extracellular_0"


In [68]:
minimum_auxotrophies = []

#strain_model.reactions.EX_cpd00007_e.lower_bound=0 # shut off O2 uptake
#strain_model.reactions.EX_cpd00011_e.lower_bound=0 # Shut off CO2 uptake
essential_threshold = 0.01
max_growth = model.slim_optimize()
consumed_dict = {}
produced_dict = {}
# exchanges that should never be shut off
# H+, H2O
exclusion_list = ['EX_cpd00067_e','EX_cpd00001_e']

# Should be performed with and without oxygen available
oxygen_exchange = 'EX_cpd00007_e'

exclude_exchange = []
minimum_auxotrophies = []
while not minimum_auxotrophies:
    print('aa')
    with model:
    #for i in range(0,2):
        # get media components that are always essential
        for reaction in exclude_exchange:
            model.reactions.get_by_id(reaction).lower_bound = 0
        media_rxns = model.medium.keys()
        current_growth = model.slim_optimize()
        deletion_results = single_media_removal(model,media_rxns)
        # Get the deletions that result in growth below the threshold and
        
        essential = [metabolite for metabolite,growthrate in deletion_results.items() if growthrate < essential_threshold*max_growth]
        
        # add the pFBA constraints and objective to the model
        add_pfba(model,fraction_of_optimum=1.0)
        pfba_sol = model.optimize()
        # extract the media fluxes only
        active = pfba_sol.fluxes.loc[abs(pfba_sol.fluxes) > 0]
        active_media = active[[rxn for rxn in model.medium.keys() if rxn in active.index]]

        # remove the essential reactions from the active_media list
        active_nonessential = active_media[[i for i in active_media.index if i not in essential]].sort_values()
        
        # filter out reactions from the exclusion list
        active_nonessential = active_nonessential[list(set(active_nonessential.index) - set(exclusion_list))]
        if len(active_nonessential[active_nonessential < 0]) > 0:
            produced = active_nonessential[active_nonessential > 0].sort_values(ascending=True)
            for metabolite in produced.index:
                if metabolite in produced_dict.keys():
                    produced_dict[metabolite].append({'production_flux':produced[metabolite],'growth_rate':current_growth})
                else:
                    produced_dict[metabolite] = [{'production_flux':produced[metabolite],'growth_rate':current_growth}]
            max_consumed = active_nonessential[active_nonessential < 0].sort_values(ascending=False).index[-1]
            #produced_dict[max_produced] = active_media[max_produced]
            consumed_dict[max_consumed] = (active_media[max_consumed],current_growth)

            print(max_consumed,model.reactions.get_by_id(max_consumed).name,active_media[max_consumed],current_growth)
            #print(max_produced,model.reactions.get_by_id(max_produced).name,active_media[max_produced],current_growth)
            # shut off the exchange reaction for the max consumed metabolite
            exclude_exchange.append(max_consumed)
        else:
            minimum_auxotrophies = essential
            minimum_growth = current_growth

aa
EX_cpd00276_b EX_cpd00276_b -1000.0000000001442 13.317220518370382
aa
EX_cpd00307_b EX_cpd00307_b -91.46817831650695 7.869753556918997
aa
EX_cpd00588_b EX_cpd00588_b -5.227121881741488 5.9672801127733885
aa
EX_cpd00314_b EX_cpd00314_b -5.227121881741793 5.967280112773404
aa


In [76]:
model.metabolites.cpd00314_e0

Metabolite identifier,cpd00314_e0
Name,D_Mannitol_e0
Memory address,0x07f97b5da5550
Formula,
Compartment,e0
In 2 reaction(s),"rxn05617_c0, EX_cpd00314_e0"


In [77]:
active_nonessential

EX_cpd00528_b     60.600067
EX_cpd00082_b    175.889414
EX_cpd00106_b      0.709277
EX_cpd11416_b      5.583035
EX_cpd00129_b    130.197147
EX_cpd00001_b    324.106658
EX_cpd00027_b     24.719743
EX_cpd00007_b      0.147774
Name: fluxes, dtype: float64

In [60]:
active_nonessential

EX_cpd11416_b      72.725149
EX_cpd00154_b     100.212232
EX_cpd00001_b    1000.000000
EX_cpd00011_b     713.854369
Name: fluxes, dtype: float64

In [35]:
def single_media_removal(model,exchange_id_list):
    '''
    Perform single metabolite removals from the media.
    In contrast to a reaction deletion, this only sets
    the lower bound for the reaction to 0 but still
    allows secretion of the metabolite.
    
    The exchange_id_list must be a list of reaction ids
    for the corresponding exchange reactions.
    '''
    exchange_to_growthrate = {}
    for exchange_id in exchange_id_list:
        with model:
            model.reactions.get_by_id(exchange_id).lower_bound = 0.0
            exchange_to_growthrate[exchange_id] = model.slim_optimize()
    return(exchange_to_growthrate)

In [11]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as colors
# set the colormap and centre the colorbar - from http://chris35wills.github.io/matplotlib_diverging_colorbar/
class MidpointNormalize(colors.Normalize):
    """
    Normalise the colorbar so that diverging bars work there way either side from a prescribed midpoint value)

    e.g. im=ax1.imshow(array, norm=MidpointNormalize(midpoint=0.,vmin=-100, vmax=100))
    """
    def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
        self.midpoint = midpoint
        colors.Normalize.__init__(self, vmin, vmax, clip)

    def __call__(self, value, clip=None):
        # I'm ignoring masked values and all kinds of edge cases to make a
        # simple example...
        x, y = [self.vmin, self.midpoint, self.vmax], [0, 0.5, 1]
        return np.ma.masked_array(np.interp(value, x, y), np.isnan(value))